In [1]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
SEED = 1013
np.random.seed(SEED)
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 
from stance_utils import *
#from parameters import *
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout,Concatenate,Dense, Embedding, LSTM, SpatialDropout1D, Flatten, GRU, Bidirectional, Conv1D, Input,MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
stemmer = PorterStemmer()
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stopwords_english = stopwords.words('english')
from sklearn.preprocessing import LabelEncoder

In [2]:
classes = {'FAVOR': np.array([1, 0, 0]), 'AGAINST': np.array([0, 1, 0]), 'NONE': np.array([0, 0, 1])}
classes_ = np.array(['FAVOR', 'AGAINST', 'NONE'])

In [3]:
train_data_file_m = '/data/parush/stance_mohammed/train.txt'
test_data_file_m = '/data/parush/stance_mohammed/test.txt'
TARGETS_m = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]


train_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/train.txt'
test_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/test.txt'
TARGETS_s = ['god','healthcare','guns','gayRights','abortion', 'creation']


train_data_file_q = '/data/parush/Data_MPCHI/train.txt'
test_data_file_q = '/data/parush/Data_MPCHI/test.txt'
TARGETS_q = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
      'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
      'Should women take HRT post-menopause?']


In [1]:
def train_and_test(train_file, test_file, target):
    
    sentence_maxlen = 0
    x_train = []
    y_train = []

    
    with open(train_file, 'r') as trainfile:
        for line in trainfile:
            
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            #if line[0].strip() != 'ID' and line[1].strip() == t:
            if line[0].strip() != 'ID' and target in line[1].strip():
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_train.append(tweet)
                y_train.append(classes[line[3].strip()])
    
    with open('rich_data_feminism_athesim_abortion.txt','r') as newfile:
        for line in newfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            tweet = line[0]
            tweet = process_tweet(tweet)
            if len(tweet) > sentence_maxlen:
                sentence_maxlen = len(tweet)
            x_train.append(tweet)
            y_train.append(classes[line[1].strip()])
    
                               
    
    x_test = []
    y_test = []
    with open(test_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            #if line[0] != 'ID' and line[1] == t:
            if line[0] != 'ID' and target in line[1].strip():
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_test.append(tweet)
                y_test.append(classes[line[3].strip()])


    
    return x_train, y_train, x_test, y_test, sentence_maxlen

In [5]:
train_file = train_data_file_m
test_file = test_data_file_m
target = TARGETS_m[2]

In [6]:
x_train, y_train, x_test, y_test, sentence_maxlen = train_and_test(train_file,test_file,target)

In [7]:
len(x_train)

1046

In [8]:
vocabulary = build_vocab(x_train)
vocab_size = len(vocabulary)
print("Total words in vocab are",vocab_size)

Total words in vocab are 3140


In [9]:

embeddings_weights = get_embeddings('wikipedia',300,vocabulary)



In [10]:
for i in range(len(x_train)):
    tweet_tensor = tweet_to_tensor(x_train[i], vocabulary)
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_train[i] = tweet_tensor
for i in range(len(x_test)):
    tweet_tensor = tweet_to_tensor(x_test[i], vocabulary)
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_test[i] = tweet_tensor

In [11]:
x_train = np.array(x_train)

x_test = np.array(x_test)

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [12]:
x_train

array([[   3,    4,    5, ...,    0,    0,    0],
       [  17,   18,   19, ...,    0,    0,    0],
       [  13,   28,   29, ...,    0,    0,    0],
       ...,
       [1895, 3136, 1892, ...,    0,    0,    0],
       [ 421, 2120,   55, ...,    0,    0,    0],
       [ 344,  166,  423, ...,    0,    0,    0]])

In [13]:
num_classes = 3
def biLSTM():
    model = Sequential()
    model.add(Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights]))
    model.add(Dropout(0.2))
    model.add(LSTM(64,return_sequences=True,dropout=0.3))
    model.add(Bidirectional(LSTM(64,dropout=0.3)))
    #model.add(Flatten())
    #add a dropout here
    model.add(Dropout(0.5))
    model.add(Dense(num_classes,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

def biLSTMCNN():
    inputs = Input(shape=(sentence_maxlen,))
    embedded_inputs = Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights])(inputs)
    embedded_inputs = Dropout(0.2)(embedded_inputs)
    lstm = Bidirectional(LSTM(64,return_sequences=True,dropout=0.3))(embedded_inputs)
    convs = []
    for each_filter_size in [3,4,5]:
        #print(rnn.shape)
        each_conv = Conv1D(100, each_filter_size, activation='relu')(lstm)
        each_conv = MaxPooling1D(sentence_maxlen-each_filter_size+1)(each_conv)
        each_conv = Flatten()(each_conv)
        #print(each_conv.shape)
        convs.append(each_conv)
        
    output = Concatenate()(convs)
    output = Dropout(0.5)(output)
    output = (Dense(3,activation='softmax'))(output)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) 
    return model

def biGRU():
    model = Sequential()
    model.add(Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights]))
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(64,return_sequences=True,dropout=0.3)))
    model.add(Bidirectional(GRU(64,dropout=0.3)))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

def biGRUCNN():
    inputs = Input(shape=(sentence_maxlen,))
    embedded_inputs = Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights])(inputs)
    embedded_inputs = Dropout(0.2)(embedded_inputs)
    rnn = Bidirectional(GRU(64,return_sequences=True,dropout=0.3))(embedded_inputs)
    convs = []
    for each_filter_size in [3,4,5]:
        #print(rnn.shape)
        each_conv = Conv1D(100, each_filter_size, activation='relu')(rnn)
        each_conv = MaxPooling1D(sentence_maxlen-each_filter_size+1)(each_conv)
        each_conv = Flatten()(each_conv)
        #print(each_conv.shape)
        convs.append(each_conv)
        
    output = Concatenate()(convs)
    output = Dropout(0.5)(output)
    output = (Dense(3,activation='softmax'))(output)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])    
    
    return model

In [14]:
model = biLSTMCNN()
print(model.summary())


2021-09-14 15:56:28.566754: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-14 15:56:30.789043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10421 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1
2021-09-14 15:56:30.790978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10421 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2021-09-14 15:56:30.792516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/t

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 18)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 18, 300)      942000      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 18, 300)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 18, 128)      186880      dropout[0][0]                    
______________________________________________________________________________________________

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
cvscores = []

for train, val in kfold.split(x_train, classes_[y_train.argmax(1)]):
    model.fit(x_train[train], y_train[train], epochs = 50, batch_size = 16, verbose=1)
    scores = model.evaluate(x_train[val], y_train[val], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

y_pred = np.round(model.predict(x_test))




Epoch 1/50


2021-09-14 15:56:31.618822: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-14 15:56:35.045159: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


53/53 [==============================] - 6s 8ms/step - loss: 1.0131 - accuracy: 0.4833
Epoch 2/50
53/53 [==============================] - 0s 6ms/step - loss: 0.8364 - accuracy: 0.6005
Epoch 3/50
53/53 [==============================] - 0s 7ms/step - loss: 0.6580 - accuracy: 0.7249
Epoch 4/50
53/53 [==============================] - 0s 6ms/step - loss: 0.4432 - accuracy: 0.8266
Epoch 5/50
53/53 [==============================] - 0s 6ms/step - loss: 0.2529 - accuracy: 0.9115
Epoch 6/50
53/53 [==============================] - 0s 6ms/step - loss: 0.1352 - accuracy: 0.9557
Epoch 7/50
53/53 [==============================] - 0s 6ms/step - loss: 0.1035 - accuracy: 0.9605
Epoch 8/50
53/53 [==============================] - 0s 6ms/step - loss: 0.1072 - accuracy: 0.9665
Epoch 9/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0873 - accuracy: 0.9677
Epoch 10/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0534 - accuracy: 0.9809
Epoch 11/50
53/53 [===========

In [16]:
print("Trained on {} +  and tested on {} \n".format(target,target),classification_report(y_test, y_pred, digits=4, labels = [0,1]))

Trained on Feminist Movement +  and tested on Feminist Movement 
               precision    recall  f1-score   support

           0     0.3218    0.4828    0.3862        58
           1     0.7362    0.6557    0.6936       183

   micro avg     0.5920    0.6141    0.6029       241
   macro avg     0.5290    0.5692    0.5399       241
weighted avg     0.6365    0.6141    0.6197       241
 samples avg     0.5193    0.5193    0.5193       241



/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# print("Trained on {} + Siamese prediction on HRT as Feminism -  and tested on {} \n".format(target,target),classification_report(y_test, y_pred, digits=4, labels = [0,1]))

# Getting test data from another datset. We will use above model to predict here.

In [18]:
new_train_file = train_data_file_m
new_test_file = test_data_file_m
new_target = TARGETS_m[4]

In [19]:

_, _, x_test_, y_test_, _ = train_and_test(new_train_file, new_test_file, new_target)

In [20]:
for i in range(len(x_test_)):
    tweet_tensor_ = tweet_to_tensor(x_test_[i], vocabulary)
    if len(tweet_tensor_) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor_)
        n_pad = [0]*diff
        tweet_tensor_ = tweet_tensor_ + n_pad
    else:
        tweet_tensor_ = tweet_tensor_[:sentence_maxlen]
    x_test_[i] = tweet_tensor_


x_test_ = np.array(x_test_)
y_test_ = np.asarray(y_test_).astype('float32')

In [21]:
y_pred_ = np.round(model.predict(x_test_))
print("Trained on {} and tested on {} \n".format(target,new_target),classification_report(y_test_, y_pred_, digits=4, labels = [0,1]))
#print("Trained on {} + Siamese prediction on HRT as Feminism -  and tested on {} \n".format(target,new_target),classification_report(y_test_, y_pred_, digits=4, labels = [0,1]))

Trained on Feminist Movement and tested on Legalization of Abortion 
               precision    recall  f1-score   support

           0     0.2899    0.4348    0.3478        46
           1     0.7610    0.6402    0.6954       189

   micro avg     0.6184    0.6000    0.6091       235
   macro avg     0.5254    0.5375    0.5216       235
weighted avg     0.6688    0.6000    0.6274       235
 samples avg     0.5036    0.5036    0.5036       235



/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
